In [1]:
import json, os, csv
import numpy as np
import pandas as pd
from scipy.stats import gmean
from IPython.display import display, HTML

In [2]:
# filepath to the file containing venue names and their categories (conf or journal)
venue_category_filename = os.path.join(os.pardir,"app","data","venue_list.csv")
venue_weight_filename = os.path.join(os.pardir,"app","data","venue_weight.csv")
filtered_papers_filename = lambda name,year: os.path.join( "filtered_papers" , 
                                              "{}_{}_filtered_papers.json".format(name,year) )
previous_weight_filename = os.path.join(os.pardir,"app","data","venue_weight_old.csv")
yearranges = range(2007,2020)

# Count paper numbers

In [3]:
paper_data = {}
def count_paper_numbers(venue):
    paper_data[venue] = {y:0 for y in yearranges}
    for year in yearranges:
        with open(filtered_papers_filename(venue, year), "r") as fh:
            papers = json.load(fh)
            paper_data[venue][year] = len(papers)
    paper_data[venue]["Total"] = sum(paper_data[venue].values())

In [4]:
with open(venue_category_filename, "r") as fh:
    reader = csv.reader(fh, delimiter=",")
    # skip header row
    next(reader)
    for row in reader:
        venue_name = row[0]
        count_paper_numbers(venue_name)

# Compute Geometric mean

One difficulty with the geometric mean is that it is undefined when the number of citations is zero. There indeed are some papers with zero citations. To handle this case, we add 1 to the number of citations before computing the geometric mean, and then subtract 1 from the mean obtained.

In [5]:
def compute_venue_weight(venue):
    citation_numbers = []
    paper_numbers = []
    no_matching_paper_count =0
    for year in yearranges:
        with open(filtered_papers_filename(venue, year), "r") as fh:
            papers = json.load(fh)
            paper_numbers.append(len(papers))
            for p in papers:
                mag_matching_papers = p["MAG papers"]
                if len(mag_matching_papers) == 0: # no matching paper from MAG
                    no_matching_paper_count += 1
                    continue
                if len(mag_matching_papers) > 1: # more than one matching papers --> take max cc number
                    citation_count = max([cc["CitationCount"] for cc in mag_matching_papers])
                else:
                    citation_count = mag_matching_papers[0]["CitationCount"]
                citation_numbers.append(1+citation_count)
#     print(venue, citation_numbers)
    paper_data[venue]["NoMatch"] = no_matching_paper_count
    paper_data[venue]["p_NoMatch(%)"] = float(no_matching_paper_count)/float(paper_data[venue]["Total"])*100.0
    paper_data[venue]["GeoMean"] = gmean(citation_numbers)-1
    paper_data[venue]["AriMean"] = np.mean(citation_numbers)-1

In [6]:
with open(venue_category_filename, "r") as fh:
    reader = csv.reader(fh, delimiter=",")
    # skip header row
    next(reader)
    for row in reader:
        venue_name = row[0]
        compute_venue_weight(venue_name)

In [7]:
# read previous weight
with open(previous_weight_filename, "r") as fh:
    reader = csv.reader(fh, delimiter=",")
    # skip header row
    next(reader)
    for row in reader:
        venue_name = row[0]
        pre_amean = row[1]
        pre_gmean = row[2]
        paper_data[venue_name]["old_GeoMean"] = float(pre_gmean)
        paper_data[venue_name]["old_AriMean"] = float(pre_amean)

In [8]:
df = pd.DataFrame(data=paper_data).T
# df.columns.tolist()

In [9]:
def cell_color(val):
    color = 'black'
    if val == 0:
        color = 'red'
    return 'color: %s' % color

df = df.astype({y:int for y in yearranges})
df = df.astype({"Total":int, "NoMatch":int, "AriMean":float, "GeoMean":float, "p_NoMatch(%)":float})
pd.set_option('display.max_rows', len(df))
print("NoMatch: No matching paper from MAG")
print("p_NoMatch(%): 100*NoMatch/Total")
coverage = 100*sum([pv["NoMatch"] for pv in paper_data.values()])/sum([pv["Total"] for pv in paper_data.values()])
print("{}% of the papers did not match from title search".format(coverage))

NoMatch: No matching paper from MAG
p_NoMatch(%): 100*NoMatch/Total
4.729900901572163% of the papers did not match from title search


In [10]:
df.style.applymap(cell_color)

In [11]:
df.to_csv(venue_weight_filename, columns=["AriMean", "GeoMean"])
